<a href="https://colab.research.google.com/github/omorros/SmartFoodAI/blob/main/SmartFoodAI_ImageTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install -q kaggle tensorflow

In [21]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
tf.config.list_physical_devices('GPU')

TensorFlow version: 2.19.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [1]:
from google.colab import files
uploaded = files.upload()

import os
os.makedirs('/root/.kaggle', exist_ok=True)
os.replace('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 0o600)


Saving kaggle.json to kaggle.json


In [24]:
!kaggle datasets download -d moltean/fruits

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
fruits.zip: Skipping, found more recently modified local copy (use --force to force download)


In [25]:
import zipfile

with zipfile.ZipFile("fruits.zip", "r") as zip_ref:
    zip_ref.extractall("fruits_dataset")

In [26]:
import os

for root, dirs, files in os.walk("fruits_dataset"):
    print(root)
    break

fruits_dataset
